In [1]:
import os
import json
import pandas as pd
from PIL import Image

from ultralytics import YOLO

from src.image_processing import process_images
from src.model_utils import predict_boxes

In [2]:
model = YOLO("runs/detect/train/weights/best.pt")

In [3]:
def crop_images_and_generate_csv(image_folder, output_folder, csv_path):
    """
    Crop images based on the 'poly_coord' in JSON files and generate a CSV with crop filename and label.

    Args:
        image_folder (str): Path to the folder containing images and JSON files.
        output_folder (str): Path to the folder where cropped images will be saved.
        csv_path (str): Path to save the output CSV file.

    Returns:
        None
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List to store the CSV data
    csv_data = []

    for file_name in os.listdir(image_folder):
        if file_name.endswith(".json"):
            # Corresponding image file
            image_name = file_name.replace(".json", ".jpg")
            image_path = os.path.join(image_folder, image_name)
            json_path = os.path.join(image_folder, file_name)

            # Check if the image exists
            if not os.path.exists(image_path):
                print(f"Image {image_name} not found for {file_name}. Skipping.")
                continue

            # Open the image
            image = Image.open(image_path)

            # Load JSON data
            with open(json_path, 'r') as json_file:
                data = json.load(json_file)

            # Process each license plate in the JSON
            for idx, lp in enumerate(data.get("lps", [])):
                poly_coord = lp.get("poly_coord", [])
                lp_id = lp.get("lp_id", "Unknown")

                if len(poly_coord) != 4:
                    print(f"Invalid polygon coordinates in {file_name}. Skipping.")
                    continue

                # Calculate bounding box from polygon coordinates
                x_coords = [coord[0] for coord in poly_coord]
                y_coords = [coord[1] for coord in poly_coord]
                bbox = (min(x_coords), min(y_coords), max(x_coords), max(y_coords))

                # Crop the image
                cropped_image = image.crop(bbox)

                # Save the cropped image
                output_image_name = f"{os.path.splitext(image_name)[0]}_crop_{idx + 1}.jpg"
                output_image_path = os.path.join(output_folder, output_image_name)
                cropped_image.save(output_image_path)

                # Add entry to CSV data
                csv_data.append({"filename": output_image_name, "label": lp_id})

                print(f"Cropped image saved: {output_image_path}")

    # Create and save the CSV
    df = pd.DataFrame(csv_data)
    df.to_csv(csv_path, index=False)
    print(f"CSV file saved: {csv_path}")

In [ ]:
splits = ['train', 'test', 'val']

for split in splits:
    if split == 'train':
        image_folder = "data/train"
        cropped_output_folder = "cropped_images/train"
        processed_output_folder = "cropped_images_processed/train"
        csv_path = "labels.csv"
        
        crop_images_and_generate_csv(image_folder, cropped_output_folder, csv_path)
        process_images(cropped_output_folder, processed_output_folder)
    
    else:
        input_folder = f"cropped_images/{split}"
        processed_output_folder = f"cropped_images_processed/{split}"
        
        preds = predict_boxes(model, split=split, get_cropped_images=True)
        
        process_images(input_folder, processed_output_folder)


Cropped image saved: cropped_images/train/01599_crop_1.jpg
Cropped image saved: cropped_images/train/00222_crop_1.jpg
Cropped image saved: cropped_images/train/00672_crop_1.jpg
Cropped image saved: cropped_images/train/01960_crop_1.jpg
Cropped image saved: cropped_images/train/00388_crop_1.jpg
Cropped image saved: cropped_images/train/00367_crop_1.jpg
Cropped image saved: cropped_images/train/01126_crop_1.jpg
Invalid polygon coordinates in 01608.json. Skipping.
Cropped image saved: cropped_images/train/01258_crop_1.jpg
Cropped image saved: cropped_images/train/00449_crop_1.jpg
Cropped image saved: cropped_images/train/00449_crop_2.jpg
Cropped image saved: cropped_images/train/00625_crop_1.jpg
Cropped image saved: cropped_images/train/01937_crop_1.jpg
Cropped image saved: cropped_images/train/00275_crop_1.jpg
Cropped image saved: cropped_images/train/01034_crop_1.jpg
Cropped image saved: cropped_images/train/01464_crop_1.jpg
Cropped image saved: cropped_images/train/01464_crop_2.jpg
Inv

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 7 extraneous bytes before marker 0xd9
Corrupt JPEG data: 3 extraneous bytes before marker 0xd9
Corrupt JPEG data: 5 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 7 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment


Deskewed image saved: cropped_images_processed/train/00657_crop_1.jpg
Processed and saved: cropped_images_processed/train/00657_crop_1.jpg
Processing 01085_crop_2.jpg...
Deskewed image saved: cropped_images_processed/train/01085_crop_2.jpg
Processed and saved: cropped_images_processed/train/01085_crop_2.jpg
Processing 00774_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/00774_crop_1.jpg
Processed and saved: cropped_images_processed/train/00774_crop_1.jpg
Processing 01568_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/01568_crop_1.jpg
Processed and saved: cropped_images_processed/train/01568_crop_1.jpg
Processing 00987_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/00987_crop_1.jpg
Processed and saved: cropped_images_processed/train/00987_crop_1.jpg
Processing 01715_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/01715_crop_1.jpg
Processed and saved: cropped_images_processed/train/01715_crop_1.jpg
Processing 

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 5 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 4 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 5 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segm

Processed and saved: cropped_images_processed/train/01136_crop_1.jpg
Processing 00086_crop_2.jpg...
Deskewed image saved: cropped_images_processed/train/00086_crop_2.jpg
Processed and saved: cropped_images_processed/train/00086_crop_2.jpg
Processing 00074_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/00074_crop_1.jpg
Processed and saved: cropped_images_processed/train/00074_crop_1.jpg
Processing 01268_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/01268_crop_1.jpg
Processed and saved: cropped_images_processed/train/01268_crop_1.jpg
Processing 01777_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/01777_crop_1.jpg
Processed and saved: cropped_images_processed/train/01777_crop_1.jpg
Processing 01015_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/01015_crop_1.jpg
Processed and saved: cropped_images_processed/train/01015_crop_1.jpg
Processing 00721_crop_1.jpg...
Deskewed image saved: cropped_images_processed/tra

...

Corrupt JPEG data: 2 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
Corrupt JPEG data: 4 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 2 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 4 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 3 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: pr

Deskewed image saved: cropped_images_processed/train/00565_crop_1.jpg
Processed and saved: cropped_images_processed/train/00565_crop_1.jpg
Processing 00718_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/00718_crop_1.jpg
Processed and saved: cropped_images_processed/train/00718_crop_1.jpg
Processing 01504_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/01504_crop_1.jpg
Processed and saved: cropped_images_processed/train/01504_crop_1.jpg
Processing 00346_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/00346_crop_1.jpg
Processed and saved: cropped_images_processed/train/00346_crop_1.jpg
Processing 00424_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/00424_crop_1.jpg
Processed and saved: cropped_images_processed/train/00424_crop_1.jpg
Processing 01327_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/01327_crop_1.jpg
Processed and saved: cropped_images_processed/train/01327_crop_1.jpg
Processing 

Corrupt JPEG data: 4 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 4 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 3 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: pr

Deskewed image saved: cropped_images_processed/train/01786_crop_1.jpg
Processed and saved: cropped_images_processed/train/01786_crop_1.jpg
Processing 00914_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/00914_crop_1.jpg
Processed and saved: cropped_images_processed/train/00914_crop_1.jpg
Processing 00154_crop_2.jpg...
Deskewed image saved: cropped_images_processed/train/00154_crop_2.jpg
Processed and saved: cropped_images_processed/train/00154_crop_2.jpg
Processing 01861_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/01861_crop_1.jpg
Processed and saved: cropped_images_processed/train/01861_crop_1.jpg
Processing 00800_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/00800_crop_1.jpg
Processed and saved: cropped_images_processed/train/00800_crop_1.jpg
Processing 00191_crop_1.jpg...
Deskewed image saved: cropped_images_processed/train/00191_crop_1.jpg
Processed and saved: cropped_images_processed/train/00191_crop_1.jpg
Processing 

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
